# 

# process for tabular data

In [1]:
import os


input_dir = "./../../Output_M"
output_dir = "./../../Output_table_cleaned"
os.makedirs(output_dir, exist_ok=True)


for root, _, files in os.walk(input_dir):
    for file in files:
        if file.endswith(".md"):
            input_file = os.path.join(root, file)

            # Construct corresponding output file path
            relative_path = os.path.relpath(input_file, input_dir)
            output_file = os.path.join(output_dir, relative_path)
            os.makedirs(os.path.dirname(output_file), exist_ok=True)

            with open(input_file, 'r', encoding='utf-8') as f:
                lines = f.readlines()

            cleaned_lines = []

            # for i in range (len(lines)):
            #     print(i, lines[i])

            i = 0
            while i < len(lines) - 3:
                if (lines[i].startswith("|") and lines[i+1].strip() == "" and lines[i+2].startswith("|") and lines[i+3].startswith("|-")):
                    print("Found misbehaved table header at line", i, "in", file)

                    cleaned_lines.append(lines[i])
                    i += 1

                    # skip the empty line
                    i += 1

                    cleaned_lines.append(lines[i])
                    i += 1

                    # Skip the misplaced separator 
                    # i+=1 after this if statement
                else:
                    cleaned_lines.append(lines[i])
                i += 1

            # Append remaining lines if we stopped early
            while i < len(lines):
                cleaned_lines.append(lines[i])
                i += 1



            # save the new file 
            with open(output_file, 'w', encoding='utf-8') as f:
                f.writelines(cleaned_lines)

Found misbehaved table header at line 77 in WisTMP 8026 (1011-01-30) (1).md
Found misbehaved table header at line 98 in WisTMP 8026 (1011-01-30) (1).md
Found misbehaved table header at line 144 in WisTMP 8026 (1011-01-30) (1).md
Found misbehaved table header at line 53 in WisTMP 7999 (2695-07-01).md
Found misbehaved table header at line 53 in WisTMP 7605 (1610-41-00).md
Found misbehaved table header at line 87 in WisTMP 7605 (1610-41-00).md
Found misbehaved table header at line 106 in WisTMP 7605 (1610-41-00).md
Found misbehaved table header at line 160 in WisTMP 7605 (1610-41-00).md
Found misbehaved table header at line 53 in WisTMP 7699 (1170-20-02) (1).md
Found misbehaved table header at line 162 in WisTMP 7699 (1170-20-02) (1).md
Found misbehaved table header at line 53 in WisTMP 7682 (9140-12-31) (2).md
Found misbehaved table header at line 108 in WisTMP 7682 (9140-12-31) (2).md
Found misbehaved table header at line 138 in WisTMP 7682 (9140-12-31) (2).md
Found misbehaved table hea

# Extract comment

In [8]:
import os
import re

def remove_date_author(text):
    """
    Removes date and author information from the text.
    It finds parenthesized expressions starting with "By" and removes them.
    """
    cleaned_text = re.sub(r"\s*\(By [^)]+\)", "", text)
    return cleaned_text

def extract_section_name_and_comments(text):
    """
    Extracts tuples of (section name, section comment) from the document text.
    
    Assumes that:
      - A section header is marked by a markdown header line containing a bold text
        that starts with "Section <number> - ..." (e.g., **Section 4 - Work Zone Strategies**).
      - A "Section Comments" header is used to indicate the start of a comment block.
    
    Returns:
      A list of tuples: (section name, comment block text)
    """
    lines = text.splitlines()
    current_section = None  
    results = []            
    capturing = False       
    comment_lines = []      

    for line in lines:
        section_header_match = re.match(r"^#{1,6}\s*\*\*(Section\s+\d+\s*-\s*.*?)\*\*", line)
        if section_header_match:
            current_section = section_header_match.group(1)
        
        if re.search(r"\*\*Section Comments\*\*", line):
            if capturing and comment_lines:
                comment_text = "\n".join(comment_lines).strip()
                comment_text = remove_date_author(comment_text)
                results.append((current_section, comment_text))
                comment_lines = []
            capturing = True
            continue  
        
        if capturing and re.match(r"^#{1,6}\s*", line):
            if comment_lines:
                comment_text = "\n".join(comment_lines).strip()
                comment_text = remove_date_author(comment_text)
                results.append((current_section, comment_text))
            capturing = False
            comment_lines = []
        
        if capturing:
            comment_lines.append(line)
    
    if capturing and comment_lines:
        comment_text = "\n".join(comment_lines).strip()
        comment_text = remove_date_author(comment_text)
        results.append((current_section, comment_text))
    
    return results

# Base directory for input files
input_base_dir = "./../../Output_table_cleaned"

# Recursively process all markdown files
for root, _, files in os.walk(input_base_dir):
    for file in files:
        if file.endswith(".md"):
            input_file = os.path.join(root, file)
            
            # Generate the output file name by appending "_extracted"
            output_file = os.path.join(root, file.replace(".md", "_extracted.md"))

            # Read the document text from the input file
            with open(input_file, 'r', encoding='utf-8') as f:
                document_text = f.read()

            # Extract section names and comments
            section_data = extract_section_name_and_comments(document_text)

            # Save the extracted content
            if section_data:
                with open(output_file, 'w', encoding='utf-8') as out_file:
                    for section_name, comment in section_data:
                        out_file.write(f"{section_name}\n")
                        out_file.write(f"{comment}\n")

                print(f"Processed: {input_file} -> {output_file}")


Processed: ./../../Output_table_cleaned/WisTMP 7999 (2695-07-01)/WisTMP 7999 (2695-07-01).md -> ./../../Output_table_cleaned/WisTMP 7999 (2695-07-01)/WisTMP 7999 (2695-07-01)_extracted.md
Processed: ./../../Output_table_cleaned/WisTMP 7834 (2250-14-11)/WisTMP 7834 (2250-14-11).md -> ./../../Output_table_cleaned/WisTMP 7834 (2250-14-11)/WisTMP 7834 (2250-14-11)_extracted.md
Processed: ./../../Output_table_cleaned/WisTMP 8172 (4236-02-00)/WisTMP 8172 (4236-02-00).md -> ./../../Output_table_cleaned/WisTMP 8172 (4236-02-00)/WisTMP 8172 (4236-02-00)_extracted.md
Processed: ./../../Output_table_cleaned/WisTMP 7828 (1330-29-00)/WisTMP 7828 (1330-29-00).md -> ./../../Output_table_cleaned/WisTMP 7828 (1330-29-00)/WisTMP 7828 (1330-29-00)_extracted.md
Processed: ./../../Output_table_cleaned/WisTMP 8941 (8941-05-01)/WisTMP 8941 (8941-05-01).md -> ./../../Output_table_cleaned/WisTMP 8941 (8941-05-01)/WisTMP 8941 (8941-05-01)_extracted.md
Processed: ./../../Output_table_cleaned/WisTMP 8741 (NO DESI

# Remvoe & extract comment

In [11]:
import os
import re

def remove_date_author(text):
    """Removes date and author information from the text."""
    return re.sub(r"\s*\(By [^)]+\)", "", text)

def extract_and_remove_comments(text):
    """
    Extracts section names and comment blocks, and removes the comment blocks 
    from the original text.

    Returns:
      - results: a list of tuples (section name, comment text)
      - cleaned_text: the original text with comment blocks removed
    """
    lines = text.splitlines()
    current_section = None
    results = []
    capturing = False
    comment_lines = []
    cleaned_lines = []

    for line in lines:
        # Only update current_section when we're not capturing a comment block.
        if not capturing:
            header_match = re.match(r"^#{1,6}\s*\*\*(Section\s+\d+\s*-\s*.*?)\*\*", line)
            if header_match:
                current_section = header_match.group(1)
        
        # Check for the start of a comment block.
        if re.search(r"\*\*Section Comments\*\*", line):
            # If we were already capturing (should rarely happen), flush the block.
            if capturing and comment_lines:
                comment_text = "\n".join(comment_lines).strip()
                comment_text = remove_date_author(comment_text)
                results.append((current_section, comment_text))
                comment_lines = []
            capturing = True
            # Skip this line so it is not added to cleaned text.
            continue

        # If we are capturing and we hit a new markdown header, finish capturing.
        if capturing and re.match(r"^#{1,6}\s*", line):
            if comment_lines:
                comment_text = "\n".join(comment_lines).strip()
                comment_text = remove_date_author(comment_text)
                results.append((current_section, comment_text))
            capturing = False
            comment_lines = []
            # Now process this header line as a normal line (updating current_section below).
            header_match = re.match(r"^#{1,6}\s*\*\*(Section\s+\d+\s*-\s*.*?)\*\*", line)
            if header_match:
                current_section = header_match.group(1)
            cleaned_lines.append(line)
            continue

        # If capturing, collect comment lines; otherwise, add to cleaned text.
        if capturing:
            comment_lines.append(line)
        else:
            cleaned_lines.append(line)

    # If the file ends while still capturing a comment block, flush it.
    if capturing and comment_lines:
        comment_text = "\n".join(comment_lines).strip()
        comment_text = remove_date_author(comment_text)
        results.append((current_section, comment_text))

    return results, "\n".join(cleaned_lines)


# Base directory for input files
input_base_dir = "./../../Output_M"

# Process all markdown files recursively.
for root, _, files in os.walk(input_base_dir):
    for file in files:
        if file.endswith(".md"):
            input_file = os.path.join(root, file)
            # Save the extracted comments next to the original file.
            output_file = os.path.join(root, file.replace(".md", "_extracted.md"))

            # Read the original file.
            with open(input_file, 'r', encoding='utf-8') as f:
                document_text = f.read()

            # Extract comments and get cleaned text.
            section_data, cleaned_text = extract_and_remove_comments(document_text)

            # Save the extracted comments if any were found.
            if section_data:
                with open(output_file, 'w', encoding='utf-8') as out_file:
                    for section_name, comment in section_data:
                        out_file.write(f"{section_name}\n")
                        out_file.write(f"{comment}\n")
                print(f"Extracted comments: {input_file} -> {output_file}")

            # Overwrite the original file with the cleaned text (without comment blocks).
            with open(input_file, 'w', encoding='utf-8') as f:
                f.write(cleaned_text)
            print(f"Updated original file: {input_file}")


Updated original file: ./../../Output_M/WisTMP 8026 (1011-01-30) (1)/WisTMP 8026 (1011-01-30) (1).md
Extracted comments: ./../../Output_M/WisTMP 7999 (2695-07-01)/WisTMP 7999 (2695-07-01).md -> ./../../Output_M/WisTMP 7999 (2695-07-01)/WisTMP 7999 (2695-07-01)_extracted.md
Updated original file: ./../../Output_M/WisTMP 7999 (2695-07-01)/WisTMP 7999 (2695-07-01).md
Updated original file: ./../../Output_M/WisTMP 7605 (1610-41-00)/WisTMP 7605 (1610-41-00).md
Updated original file: ./../../Output_M/WisTMP 7699 (1170-20-02) (1)/WisTMP 7699 (1170-20-02) (1).md
Updated original file: ./../../Output_M/WisTMP 8551 (5808-00-06) (1)/WisTMP 8551 (5808-00-06) (1).md
Updated original file: ./../../Output_M/WisTMP 7682 (9140-12-31) (2)/WisTMP 7682 (9140-12-31) (2).md
Extracted comments: ./../../Output_M/WisTMP 7834 (2250-14-11)/WisTMP 7834 (2250-14-11).md -> ./../../Output_M/WisTMP 7834 (2250-14-11)/WisTMP 7834 (2250-14-11)_extracted.md
Updated original file: ./../../Output_M/WisTMP 7834 (2250-14-11)